In [7]:
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# Functions

In [2]:
def count_chars(text):
    return len(text)


def count_words(text):
    return len(text.split())


def count_capital_chars(text):
    count=0
    for i in text:
        if i.isupper():
            count+=1
    return count


def count_capital_words(text):
    return sum(map(str.isupper,text.split()))


def count_punctuations(text):
    punctuations= string.punctuation
    d=dict()
    for i in punctuations:
        d[str(i)+' count']=text.count(i)
    return d 


# def count_words_in_quotes(text):
#     x = re.findall("'.'|"."", text)
#     count=0
#     if x is None:
#         return 0
#     else:
#         for i in x:
#             t=i[1:-1]
#             count+=count_words(t)
#         return count
    

def count_sent(text):
    return len(nltk.sent_tokenize(text))


def count_unique_words(text):
    return len(set(text.split()))


def count_htags(text):
    x = re.findall(r'(#w[A-Za-z0-9]*)', text)
    return len(x) 


def count_mentions(text):
    x = re.findall(r'(@w[A-Za-z0-9]*)', text)
    return len(x)


def count_stopwords(text):
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(text)
    stopwords_x = [w for w in word_tokens if w in stop_words]
    return len(stopwords_x)

In [3]:
data = '/Users/fttiunjani/gemastik/practice/dataset/dataset-idsa-master/labelled-sentiment-copy.csv'
df = pd.read_csv(data)

In [4]:
df.head()

,sentimen,Tweet
0,-1,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...
1,-1,kita lanjutkan saja diam ini hingga kau dan ak...
2,1,doa rezeki tak putus inna haa zaa larizquna ma...
3,1,makasih loh ntar kita bagi hasil aku 99 9 sisa...
4,-1,aku tak faham betul jenis orang malaysia yang ...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10806 entries, 0 to 10805
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentimen  10806 non-null  int64 
 1   Tweet     10806 non-null  object
dtypes: int64(1), object(1)
memory usage: 169.0+ KB


In [8]:
df['char_count'] = df["Tweet"].apply(lambda x:count_chars(x))
df['word_count'] = df["Tweet"].apply(lambda x:count_words(x))
df['sent_count'] = df["Tweet"].apply(lambda x:count_sent(x))
df['capital_char_count'] = df["Tweet"].apply(lambda x:count_capital_chars(x))
df['capital_word_count'] = df["Tweet"].apply(lambda x:count_capital_words(x))
# df['quoted_word_count'] = df["tweet"].apply(lambda x:count_words_in_quotes(x))
df['stopword_count'] = df["Tweet"].apply(lambda x:count_stopwords(x))
df['unique_word_count'] = df["Tweet"].apply(lambda x:count_unique_words(x))
df['htag_count'] = df["Tweet"].apply(lambda x:count_htags(x))
df['mention_count'] = df["Tweet"].apply(lambda x:count_mentions(x))
df['punct_count'] = df["Tweet"].apply(lambda x:count_punctuations(x))
df['avg_wordlength'] = df['char_count']/df['word_count']
df['avg_sentlength'] = df['word_count']/df['sent_count']
df['unique_vs_words'] = df['unique_word_count']/df['word_count']
df['stopwords_vs_words'] = df['stopword_count']/df['word_count']

In [9]:
# Calculating average word length
df['avg_wordlength'] = df['char_count']/df['word_count']

# Calculating average sentence length
df['avg_sentlength'] = df['word_count']/df['sent_count']

# unique words vs word count feature
df['unique_vs_words'] = df['unique_word_count']/df['word_count']

# Stopwords count vs words counts feature
df['stopwords_vs_words'] = df['stopword_count']/df['word_count']

In [10]:
df.head()

,sentimen,Tweet,char_count,word_count,sent_count,capital_char_count,capital_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words
0,-1,lagu bosan apa yang aku save ni huhuhuhuhuhuhu...,65,8,1,0,0,0,8,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",8.125000,8.0,1.00,0.0
1,-1,kita lanjutkan saja diam ini hingga kau dan ak...,103,16,1,0,0,0,16,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",6.437500,16.0,1.00,0.0
2,1,doa rezeki tak putus inna haa zaa larizquna ma...,114,20,1,0,0,0,20,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.700000,20.0,1.00,0.0
3,1,makasih loh ntar kita bagi hasil aku 99 9 sisa...,59,12,1,0,0,0,12,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",4.916667,12.0,1.00,0.0
4,-1,aku tak faham betul jenis orang malaysia yang ...,107,20,1,0,0,0,17,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.350000,20.0,0.85,0.0


In [40]:
to_replace = dict(zip([-1, 0, 1],['negatif', 'netral', 'positif']))
df.replace({'sentimen':to_replace}, inplace=True)
df.head()

,sentimen,Tweet,char_count,word_count,sent_count,capital_char_count,capital_word_count,stopword_count,unique_word_count,htag_count,mention_count,punct_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words
0,negatif,agu bosan apa yang aku save ni huhuhuhuhuhuhuh...,65,8,1,0,0,0,8,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",8.125000,8.0,1.00,0.0
1,negatif,ta lanjutkan saja diam ini hingga kau dan aku ...,103,16,1,0,0,0,16,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",6.437500,16.0,1.00,0.0
2,positif,doa rezeki tak putus inna haa zaa larizquna ma...,114,20,1,0,0,0,20,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.700000,20.0,1.00,0.0
3,positif,makasih loh ntar kita bagi hasil aku sisanya...,59,12,1,0,0,0,12,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",4.916667,12.0,1.00,0.0
4,negatif,aku tak faham betul jenis orang malaysia yang ...,107,20,1,0,0,0,17,0,0,"{'! count': 0, '"" count': 0, '# count': 0, '$ ...",5.350000,20.0,0.85,0.0


In [41]:
# General Pre-processing 
def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'
def preprocess(sent):
    sent = remove_users(sent)
    sent = remove_links(sent)
    sent = sent.lower() # lower case
    sent = re.sub('['+my_punctuation + ']+', ' ', sent) # strip punctuation
    sent = re.sub('\s+', ' ', sent) #remove double spacing
    sent = re.sub('([0-9]+)', '', sent) # remove numbers
    sent_token_list = [word for word in sent.split(' ')]
    sent = ' '.join(sent_token_list)
    return sent

In [42]:
df.Tweet = df.Tweet.apply(lambda x: preprocess(x))

In [43]:
# Using TF-IDF
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(df['Tweet']).toarray()
print(vectorized.shape)

(10806, 22123)


In [44]:
df_tfidf = pd.DataFrame(vectorized)

In [45]:
df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,22113,22114,22115,22116,22117,22118,22119,22120,22121,22122
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
features = [feature for feature in df.columns if feature !='Tweet' and feature !='sentimen' and feature != 'punct_count']
features

['char_count',
 'word_count',
 'sent_count',
 'capital_char_count',
 'capital_word_count',
 'stopword_count',
 'unique_word_count',
 'htag_count',
 'mention_count',
 'avg_wordlength',
 'avg_sentlength',
 'unique_vs_words',
 'stopwords_vs_words']

In [47]:
X = pd.merge(df_tfidf, df[features], left_index=True, right_index=True)
y = df['sentimen']

In [48]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,capital_char_count,capital_word_count,stopword_count,unique_word_count,htag_count,mention_count,avg_wordlength,avg_sentlength,unique_vs_words,stopwords_vs_words
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,8,0,0,8.125000,8.0,1.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,16,0,0,6.437500,16.0,1.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,20,0,0,5.700000,20.0,1.00,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,12,0,0,4.916667,12.0,1.00,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,17,0,0,5.350000,20.0,0.85,0.0


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=123)

In [36]:
type(X_train)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8644 entries, 4940 to 3582
Columns: 22131 entries, 0 to stopwords_vs_words
dtypes: float64(22122), int64(9)
memory usage: 1.4 GB


In [50]:
print(f'tweet length for train = {len(X_train)}, sentimen length for train = {len(y_train)}')
print(f'tweet length for test = {len(X_test)}, sentimen length for train = {len(y_test)}')

tweet length for train = 8644, sentimen length for train = 8644
tweet length for test = 2162, sentimen length for train = 2162


In [51]:
rf_classifier = RandomForestClassifier(n_estimators = 1000, min_samples_split = 15, random_state = 42)
rf_classifier.fit(X_train, y_train)
rf_prediction = rf_classifier.predict(X_test)

In [52]:
print("Accuracy => ", round(accuracy_score(rf_prediction, y_test)*100, 2))
print("\nRandom Forest Classifier results: \n")
print(classification_report(y_test, rf_prediction, target_names = ['negatif', 'netral', 'positif']))

Accuracy =>  56.34

Random Forest Classifier results: 

              precision    recall  f1-score   support

     negatif       0.65      0.22      0.33       569
      netral       0.55      0.91      0.69      1068
     positif       0.60      0.24      0.34       525

    accuracy                           0.56      2162
   macro avg       0.60      0.45      0.45      2162
weighted avg       0.59      0.56      0.51      2162



In [53]:
import joblib

In [55]:
# save
joblib.dump(rf_classifier, "random_forest_sentimen_classifier.joblib")

# load
# loaded_rf = joblib.load("my_random_forest.joblib")

['random_forest_sentimen_classifier.joblib']